## Очистка текста из Wiki с помощью Yandex GPT

Полученные тексты из Wiki можно дополнительно очистить с помощью большой языковой модели. Учитывая достаточно большой объем информации, эта процедура будет существенной с точки зрения стоимости, поэтому лучше использовать асинхронный режим работы YandexGPT.

Используем версию `yandex_chain` старше 0.0.10, которая поддерживает аснхронный режим работы.

In [5]:
from yandex_chain import YandexLLM, YandexGPTModel

llm = YandexLLM(config="config.json",model=YandexGPTModel.LiteRC,max_tokens=6000)

Отработаем промпт на каком-то одном примере:

In [ ]:
with open('content/text/Аберфорт Дамблдор.txt',encoding='utf-8') as f:
    text = f.read()

prompt = """
Ты - редактор, готовящий материалы о Гарри Поттере для базы знаний чат-бота. Тебе на вход в 
тройных обратных кавычках подаётся текст, извлечённый из Wikipedia, содержащий какой-то мусор, 
повторения текста и другие ошибки форматирования. Твоя задача переписать этот текст в виде
аккуратного связного текста в текстовом формате, сохраняя все смысловые элементы текста без
изменений.
Текст: ```{}```
"""

response = llm.invoke(prompt.format(text))
print(response)

Теперь напишем цикл, который будет брать файлы из директории `text` и планировать их для обработки в асинхронном режиме. Для этого создадим словарь, в котором ключами будут идентификаторы запросов, а значениями - имена файлов. Ограничим количество файлов - 1000 за раз.

In [41]:
import os
from tqdm.auto import tqdm
import json

c = 1000
jobs = {}
pb = tqdm(total=c)
for fn in os.listdir('content/text'):
    if os.path.exists(os.path.join('content/ctext',fn)):
        continue
    else:
        pb.update(1)
        c-=1
        with open(os.path.join('content/text',fn),encoding='utf-8') as f:
            text = f.read()
        id = llm.invokeAsync(prompt.format(text))
        jobs[id] = fn
        if c<0:
            break

with open('ids.json','w',encoding='utf-8') as f:
    json.dump(jobs,f,ensure_ascii=False)


  0%|          | 1/999 [00:02<41:40,  2.51s/it]

Теперь сделаем цикл ожидания, и все обработанные файлы будем записывать на диск в другую директорию `ctext` (clean text):

In [43]:
while True:
    for k,v in jobs.items():
        c = 0
        if v is not None:
            try:
                res = llm.checkAsyncResult(k)
            except:
                print(f"Error processing job {k}, fn={v}")
                res = "ERROR"
            if res:
                with open(os.path.join('content/ctext',v),'w',encoding='utf-8') as f:
                    f.write(res)
                print(f"Job {k} done")
                jobs[k] = None
            else:
                c+=1
    if c==0:
        break
                

Если мы поочередно запустим эти ячейки несколько раз, то все файлы по итогу окажутся обработанными.